In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE,r2_score as r2,confusion_matrix
import seaborn as sns


In [ ]:
data=pd.read_csv("/content/mission_3.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.drop_duplicates()

In [ ]:
col=["Brand","Fuel Type","Transmission"]

In [ ]:
le={}
for i in col:
  le[i]=LabelEncoder()
  data[i]=le[i].fit_transform(data[i])


In [ ]:
data.columns

In [ ]:
min=MinMaxScaler()
data[['Model Year', 'Mileage (miles)','Condition']]=min.fit_transform(data[['Model Year', 'Mileage (miles)','Condition']])

In [ ]:
x=data.drop("Resale Price",axis=1)
y=data["Resale Price"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
pred_lr=lr.predict(x_test)
print(MSE(y_test,pred_lr))
print(MAE(y_test,pred_lr))
print(r2(y_test,pred_lr))

In [ ]:
dt=DecisionTreeRegressor()
dt.fit(x_train,y_train)
pred_dt=dt.predict(x_test)
print(MSE(y_test,pred_dt))
print(MAE(y_test,pred_dt))
print(r2(y_test,pred_dt))


In [ ]:
rf=RandomForestRegressor()
rf.fit(x_train,y_train)
pred_rf=rf.predict(x_test)
print(MSE(y_test,pred_rf))
print(MAE(y_test,pred_rf))
print(r2(y_test,pred_rf))

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(x=data["Resale Price"],color='black',kde=True,bins=20)
plt.title("relation between target and feature")
plt.xlabel("Resale Price")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x="Resale Price",y="Brand",data=data,color='red')
plt.title("relation between two columns")
plt.xlabel("Resale Price")
plt.ylabel("Brand")
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(x="Resale Price",y="Mileage (miles)",data=data,color='green')
plt.title("impact column on the other column")
plt.xlabel("Resale Price")
plt.ylabel("Mileage (miles)")
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x=y_test,y=pred_lr,color='red')
sns.scatterplot(x=y_test,y=pred_dt,color="green")
sns.scatterplot(x=y_test,y=pred_rf,color='black')
plt.title("prediction")
plt.xlabel("y_test")
plt.ylabel("pred")
plt.show()

In [ ]:
params={'max_depth':[5,10,15]}
grid=GridSearchCV(DecisionTreeRegressor(),params,cv=5)
grid.fit(x_train,y_train)
print(grid.best_params_)
pred_gred_dt=grid.best_estimator_.predict(x_test)
print(MSE(y_test,pred_gred_dt))

In [ ]:
param={'max_depth':[33,44,55]}
random=RandomizedSearchCV(RandomForestRegressor(),param,cv=5)
random.fit(x_train,y_train)
print(random.best_params_)
pred_random_rf=random.best_estimator_.predict(x_test)
print(MSE(y_test,pred_random_rf))


In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x=y_test,y=pred_gred_dt,color='black')
sns.scatterplot(x=y_test,y=pred_random_rf,color='blue')
plt.xlabel("y_test")
plt.ylabel("y_pred")
plt.show()

In [ ]:
import gradio as gr

In [ ]:
data.columns

In [ ]:
def haneen(Brand,ModelY,Miles,FT,Condition,Transmission):
  try:
    inputs_data=pd.DataFrame({
        "Brand":[Brand],
        "Model Year":[ModelY],
        "Mileage (miles)":[Miles],
        "Fuel Type":[FT],
        "Condition":[Condition],
        "Transmission":[Transmission]
    })
    for i in le:
      inputs_data[i]=le[i].transform(inputs_data[i])

    inputs_data[['Model Year', 'Mileage (miles)','Condition']]=min.transform(inputs_data[['Model Year', 'Mileage (miles)','Condition']])
    result=dt.predict(inputs_data)
    print(result)
    return result[0]
  except Exception as e:
    return str(e)
gr.Interface(
    fn=haneen,
    inputs= [
        gr.Dropdown(['Ford','BMW','Honda','Toyota'],label="Brand"),
        gr.Number(label="Model Year"),
        gr.Number(label="Mileage (miles)"),
        gr.Dropdown(['Electric','Diesel','Gasoline'],label="Fuel Type"),
        gr.Number(label="Condition"),
        gr.Dropdown(['Automatic','Manual'],label="Transmission")
    ],
     outputs=gr.Number(label="prediction")
).launch()